In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Prompt Templates

Um prompt para um modelo de linguagem é um conjunto de instruções ou entradas fornecidas por um usuário para guiar a resposta do modelo, ajudando-o a entender o contexto e gerar uma saída baseada em linguagem relevante e coerente, como responder a perguntas, completar frases ou participar de uma conversa.

In [25]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model='gpt-4o-mini')

In [26]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário:
{pergunta}
''')
prompt_template

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\nResponda a seguinte pergunta do usuário:\n{pergunta}\n')

In [27]:
print(prompt_template.format(pergunta='O que é um buraco negro?'))


Responda a seguinte pergunta do usuário:
O que é um buraco negro?



In [28]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
''')
prompt_template

PromptTemplate(input_variables=['n_palavras', 'pergunta'], input_types={}, partial_variables={}, template='\nResponda a seguinte pergunta do usuário em até {n_palavras} palavras:\n{pergunta}\n')

In [29]:
print(prompt_template.format(n_palavras= 10, pergunta= 'O que é um buraco negro?'))


Responda a seguinte pergunta do usuário em até 10 palavras:
O que é um buraco negro?



In [30]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
''', partial_variables={'n_palavras': 10})

print(prompt_template.format(pergunta= 'O que é um buraco negro?'))


Responda a seguinte pergunta do usuário em até 10 palavras:
O que é um buraco negro?



## Composing prompts | Unindo múltiplos prompts

In [31]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template('''
Responda a pergunta em até {n_palavras} palavras.''')

template_lingua = PromptTemplate.from_template('''
Retorne a resposta em {lingua}.''')

template_final = (
    template_word_count
    + template_lingua
    + '\nResposta a pergunta seguinte seguindo as intruções, {pergunta}'
)

print(template_final)

input_variables=['lingua', 'n_palavras', 'pergunta'] input_types={} partial_variables={} template='\nResponda a pergunta em até {n_palavras} palavras.\nRetorne a resposta em {lingua}.\nResposta a pergunta seguinte seguindo as intruções, {pergunta}'


In [32]:
prompt = template_final.format(n_palavras= 10, lingua= 'inglês', pergunta= 'O que é uma estrela')
llm.invoke(prompt)

'? \nA star is a luminous celestial body in space.'

## Templates para Chat

In [45]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model= 'gpt-4o-mini')

chat_template = ChatPromptTemplate.from_template('Essa é a minha dúvida: {duvida}')
chat_template.format_messages(duvida= 'Quem sou eu?')

[HumanMessage(content='Essa é a minha dúvida: Quem sou eu?', additional_kwargs={}, response_metadata={})]

In [46]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', 'Olá, como vai?'),
        ('ai', 'Melhor agora! Como posso ajudá-lo?'),
        ('human', '{pergunta}')
    ]
)

chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], input_types={}, partial_variables={}, template='Você é um assistente engraçado e se chama {nome_assistente}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Olá, como vai?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Melhor agora! Como posso ajudá-lo?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='{pergunta}'), additional_kwargs={})])

In [53]:
prompt = chat_template.format_messages(nome_assistente= 'Robson', pergunta= 'Qual o seu nome?')
print(prompt)

[SystemMessage(content='Você é um assistente engraçado e se chama Robson', additional_kwargs={}, response_metadata={}), HumanMessage(content='Olá, como vai?', additional_kwargs={}, response_metadata={}), AIMessage(content='Melhor agora! Como posso ajudá-lo?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})]


In [54]:
resposta = chat.invoke(prompt)
print(resposta.content)

Eu me chamo Robson! Sou seu assistente engraçado e sempre pronto para dar uma risada. E o seu nome, qual é?


## Templates de Few-shot prompting para llm

In [2]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [3]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['pergunta', 'resposta'],
    template= 'Pergunta {pergunta} \n{resposta}'
)

print(example_prompt.format(**exemplos[0]))

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing? 
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 



In [64]:
prompt = FewShotPromptTemplate(
    examples= exemplos,
    example_prompt= example_prompt,
    suffix= 'Pergunta: {input}',
    input_variables=['input']
)
prompt

FewShotPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, examples=[{'pergunta': 'Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'}, {'pergunta': 'Quando nasceu o fundador do craigslist?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. \nEntão a resposta final é: 6 de dezembro

In [69]:
final_prompt = prompt.format(input='Qual produção é mais famosa a série de The Last of Us ou Arcane?')
print(final_prompt)

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing? 
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist? 
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington? 
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhament

In [70]:
print(llm.invoke(final_prompt))

 
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: O que é The Last of Us? 
Resposta intermediária: The Last of Us é um jogo de vídeo game que foi adaptado para uma série de televisão. 
Pergunta de acompanhamento: O que é Arcane? 
Resposta intermediária: Arcane é uma série de animação baseada no jogo League of Legends. 
Pergunta de acompanhamento: Qual das duas produções teve mais sucesso? 
Resposta intermediária: The Last of Us teve mais sucesso em termos de audiência e crítica. 
Então a resposta final é: The Last of Us. 


## Templates de Few-shot prompting para chat

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI()

In [72]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{pergunta}'),
     ('ai', '{resposta}')]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={})]


In [77]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        few_shot_template,
        ('human', '{input}')
    ]
)

prompt = prompt_template.format_messages(input= 'Qual produção é mais famosa a série de The Last of Us ou Arcane?')
prompt

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Quando nasceu o fundador do craigslist?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasce

In [79]:
resposta = chat.invoke(prompt)
print(resposta.content)

A fama de uma produção pode ser subjetiva e variar de acordo com o público e a região. No entanto, ambas as produções, "The Last of Us" e "Arcane", receberam aclamação crítica e popular.

"The Last of Us", baseada no popular videogame de mesmo nome, teve um grande impacto cultural e se tornou um fenômeno na HBO, atraindo uma audiência significativa.

"Arcane", baseada no universo de "League of Legends", também foi muito bem recebida, ganhando vários prêmios e se destacando na plataforma da Netflix, especialmente por sua animação inovadora e narrativa envolvente.

Em termos de notoriedade global, "The Last of Us" pode ser considerada mais famosa, especialmente devido ao seu impacto na cultura pop e sua base de fãs consolidada. No entanto, "Arcane" também conquistou um público considerável e se tornou um marco na animação. A resposta pode variar dependendo do critério de fama utilizado.
